Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import string
import tensorflow as tf
from keras import utils

from keras import layers
from keras import losses

Data Loading

In [58]:
dataset_dir = "stack_overflow_16k"
print(dataset_dir)

stack_overflow_16k


In [59]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['csharp', 'java', 'javascript', 'python']

In [ ]:
#Test data loading
sample_file = os.path.join(train_dir, 'csharp//0.txt')
with open(sample_file) as f:
    print(f.read())

"how to pause loop while time is ticking i have made a timer where i can set time i want to wait and then do something..so this is my short timer func:..private void sleep(int interval, action action).{.    system.windows.forms.timer mytimer = new system.windows.forms.timer();.    mytimer.interval = interval; //interval is in ms   .    mytimer.start();.    mytimer.tick += (s, e) =&gt;.    {.        action();.        mytimer.stop();.    };.}...and im using this timer in loop:..foreach (string word in words).{.   sleep(5000, ()=&gt; myaction());                                           .}...without loop timer is great but in loop it wont work because loop wont stop and wait for those 5secs. it do all stuff imediately and starts timer again and again too fast...so what im trying to find out is how to make my loop wait until time runs out and myaction() is executed. im working on forms application so all threadin sleeps wont work here. also i tried all other timers but they used too much 

In [ ]:
#Creating training dataset (80%)
batch_size = 32
seed = 42

raw_train_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [ ]:
print(raw_train_ds.take(1))

<_TakeDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [69]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Text: ", text_batch.numpy()[i])
    print("Label: ", label_batch.numpy()[i])

Text:  b'"set blank to quit on exception? i\'m using blank 3..i\'ve been looking around for an answer to this, but i haven\'t found it yet. basically, i\'m running several blank scripts into a game engine, and each script has its own entry point...i\'d rather not add try: except blocks through all of my code, so i was wondering if it\'s at all possible to tell blank to quit (or perhaps assign a custom function to that ""callback"") on finding its first error, regardless of where or what it found? ..currently, the game engine will continue after finding and hitting an error, making it more difficult than necessary to diagnose issues since running into one error may make a subsequent script not work (as it relies on variables that the error-ing script set, for example). any ideas? ..i know that i could redirect the console to a file to allow for easier scrolling, but just capturing the first error and stopping the game prematurely would be really useful...okay, a couple of extra bits of 

In [ ]:
print("Label 0 =", raw_train_ds.class_names[0])
print("Label 1 =", raw_train_ds.class_names[1])
print("Label 2 =", raw_train_ds.class_names[2])
print("Label 3 =", raw_train_ds.class_names[3])


Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [ ]:
#Creating validation dataset (20%)
raw_val_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [74]:
test_dir = os.path.join(dataset_dir, 'test')
os.listdir(test_dir)

['csharp', 'java', 'javascript', 'python']

In [75]:
raw_test_ds = utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)

Found 8000 files belonging to 4 classes.


In [82]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

Vectorize Data

In [156]:
max_features = 10000
sequence_length = 150

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [157]:
# Make a text-only dataset (without labels), then call adapt 
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [158]:
text_batch, label_batch = next(iter(raw_train_ds))
print(text_batch)
print(label_batch)

tf.Tensor(
[b'"printing by prototype not working in js i am trying to make a constructor function. then i am trying to use its prototype and printing ""peter"". but it is showing an error. ....function main(){..   var func1 = function(){. this.name = ""peter"";. this.age = 27; . this.class = ""10"";.   }..   func1.prototype.printname = function(){. console.log(this.name);.   }..   return func1; .}..var a = main();..a.printname();"\n'
 b'"get current url address by blank how can i get current url like this...for example, if the url is:..http://www.test.com/app/one.bc..i want to get the path without the address the main page so like:../app/one.bc"\n'
 b'"element.removeeventlistener(\'mousedown\', externalfunction, usecapture); is not working i need to first remove the event listener before dynamically adding more elements which also need the same event listener. i am using an external function name (not an anonymous function) and specifying the same usecapture value in both the add and r

In [122]:
first_text, first_label = text_batch[5], label_batch[5]

print("Text", first_text)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_text, first_label))


Text tf.Tensor(b'"blank strip all url parameters from string? i\'ve got an url in a string like this:..http://url.com?page=121&amp;param1=hello&amp;param2=bye#hello...the above could be something else as well meaning it will be dynamic...how could i only keep the ""main page""? in the above it would be:..http://url.com?page=121"\n', shape=(), dtype=string)
Label javascript
Vectorized review (<tf.Tensor: shape=(1, 150), dtype=int64, numpy=
array([[  16, 1769,   73,  288,  488,   31,   18,  195,  275,   32,  288,
           7,    5,   18,   48,    1,  250,  176,   33,  146,   49,   36,
         481, 1904,   11,   72,   33,    1,  176,    3,   93,  355,    2,
         153,  220,    7,    2,  250,   11,   69,    1,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,  

In [123]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  scissors
 313 --->  source
Vocabulary size: 15000


In [184]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
embedding_dim = 16
model = tf.keras.Sequential([
  layers.Embedding(max_features, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(4, activation='softmax')]) #1 in case of binary, as well as sigmoid function

model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_14 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_14     │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True) , #losses.BinaryEntropy in case of binary classes
              optimizer='adam',
              metrics=['accuracy']) #BinaryAccuracy in case of binary classes

In [191]:
epochs = 25
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3200 - loss: 1.3758 - val_accuracy: 0.4850 - val_loss: 1.3311
Epoch 2/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4911 - loss: 1.3122 - val_accuracy: 0.5800 - val_loss: 1.2310
Epoch 3/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5749 - loss: 1.2071 - val_accuracy: 0.6219 - val_loss: 1.1132
Epoch 4/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6265 - loss: 1.0969 - val_accuracy: 0.6612 - val_loss: 1.0075
Epoch 5/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6739 - loss: 0.9920 - val_accuracy: 0.6894 - val_loss: 0.9214
Epoch 6/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6982 - loss: 0.9061 - val_accuracy: 0.7156 - val_loss: 0.8524
Epoch 7/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7321 - loss: 0.8359 - val_accuracy: 0.7269 - val_loss: 0.7968
Epoch 8/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7530 - loss: 0.7752 - val_accuracy: 0.

In [192]:
loss, accuracy = model.evaluate(val_ds)

# loss, accuracy = model.evaluate(test_ds)


print("Loss (Validation Dataset): ", loss)
print("Accuracy (Validation Dataset): ", accuracy)

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8035 - loss: 0.5438
Loss (Validation Dataset):  0.5151147842407227
Accuracy (Validation Dataset):  0.8106250166893005


In [ ]:
loss, accuracy = model.evaluate(test_ds)


print("Loss (Test Dataset): ", loss)
print("Accuracy (Test Dataset): ", accuracy)

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7802 - loss: 0.5558
Loss (Test Dataset):  0.5151147842407227
Accuracy (Test Dataset):  0.777999997138977


In [200]:
example = """from pyspark.sql.functions import concat, to_date, expr, when

from pyspark.sql.types import DoubleType


df = (
spark.read.option("header", "True")
.option("delimiter", ",")
.option("inferSchema", "True")
.csv("dbfs:/databricks-datasets/airlines/part-00000")
)



#dff = df.withColumn("DayofMonthFormatted", 
when(df.DayofMonth.isin([1,2,3,4,5,6,7,8,9]), "Yes").otherwise("No"))

dff = df.withColumn("DayofMonthFormatted", 
when(df.DayofMonth.isin([1,2,3,4,5,6,7,8,9]), "0" + 
str(df.DayofMonth)).otherwise(df.DayofMonth))
    
display(dff['DayofMonth', 'DayofMonthFormatted'])



#dff =  df.withColumn('DayDate', 
to_date(concat('Year','Month','DayofMonth'),'yyyyMMdd'))

#display(dff)"""
examples = tf.constant([
    example
]
)
model.predict(examples)

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_14_1/Cast:0", shape=(1,), dtype=float32). Expected shape (None, None), but input has incompatible shape (1,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1,), dtype=string)
  • training=False
  • mask=None

In [201]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('softmax')
])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
metrics = export_model.evaluate(raw_test_ds, return_dict=True)
print(metrics)

c:\Users\Qiblatain\Documents\Coding\Python Projects\Algorithms\mlenv\Lib\site-packages\keras\src\backend\tensorflow\nn.py:635: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7790 - loss: 1.0077
{'accuracy': 0.777999997138977, 'loss': 1.0107349157333374}


In [209]:
examples = tf.constant([
    """from pyspark.sql.functions import concat, to_date, expr, when

from pyspark.sql.types import DoubleType


df = (
spark.read.option("header", "True")
.option("delimiter", ",")
.option("inferSchema", "True")
.csv("dbfs:/databricks-datasets/airlines/part-00000")
)



#dff = df.withColumn("DayofMonthFormatted", 
when(df.DayofMonth.isin([1,2,3,4,5,6,7,8,9]), "Yes").otherwise("No"))

dff = df.withColumn("DayofMonthFormatted", 
when(df.DayofMonth.isin([1,2,3,4,5,6,7,8,9]), "0" + 
str(df.DayofMonth)).otherwise(df.DayofMonth))
    
display(dff['DayofMonth', 'DayofMonthFormatted'])



#dff =  df.withColumn('DayDate', 
to_date(concat('Year','Month','DayofMonth'),'yyyyMMdd'))

#display(dff)""",
"JavaScript is a dynamic programming language used for creating interactive websites and web applications, enabling client-side functionality."

])

In [ ]:
import numpy as np
predictions = export_model.predict(examples)
predicted_classes = np.argmax(predictions, axis=1)  # Get the class with the highest probability

classes = [class_name for class_name in raw_test_ds.class_names]
predicted_class_names = [classes[i] for i in predicted_classes]

# Print the predicted class names
for i in range(len(predicted_class_names)):
    print("Text: "+ examples[i] + "\nClass: " + predicted_class_names[i])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
tf.Tensor(b'Text: from pyspark.sql.functions import concat, to_date, expr, when\n\nfrom pyspark.sql.types import DoubleType\n\n\ndf = (\nspark.read.option("header", "True")\n.option("delimiter", ",")\n.option("inferSchema", "True")\n.csv("dbfs:/databricks-datasets/airlines/part-00000")\n)\n\n\n\n#dff = df.withColumn("DayofMonthFormatted", \nwhen(df.DayofMonth.isin([1,2,3,4,5,6,7,8,9]), "Yes").otherwise("No"))\n\ndff = df.withColumn("DayofMonthFormatted", \nwhen(df.DayofMonth.isin([1,2,3,4,5,6,7,8,9]), "0" + \nstr(df.DayofMonth)).otherwise(df.DayofMonth))\n    \ndisplay(dff[\'DayofMonth\', \'DayofMonthFormatted\'])\n\n\n\n#dff =  df.withColumn(\'DayDate\', \nto_date(concat(\'Year\',\'Month\',\'DayofMonth\'),\'yyyyMMdd\'))\n\n#display(dff)\nClass: python', shape=(), dtype=string)
tf.Tensor(b'Text: JavaScript is a dynamic programming language used for creating interactive websites and web applications, enabling client-side functionality.\nClass: javas